In [1]:
#load the data
data = open('flare').read()
#移除换行符
data = data.replace('\n','').replace('\r','')
print(data)

flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained his phd in Australia. Australia is a country in the Southern Hemisphere.flare is a teacher in ai industry. He obtained h

In [2]:
#字符去重处理
letters = list(set(data))
print(letters)
num_letters = len(letters)
print(num_letters)

['S', 't', 'H', 'c', 'u', 'm', 'i', 'f', 'e', 'A', ' ', '.', 's', 'a', 'l', 'b', 'h', 'r', 'y', 'd', 'o', 'p', 'n']
23


In [3]:
#建立字典
#int to char
int_to_char = {a:b for a,b in enumerate(letters)}
print(int_to_char)
#char to int
char_to_int = {b:a for a,b in enumerate(letters)}
print(char_to_int)

{0: 'S', 1: 't', 2: 'H', 3: 'c', 4: 'u', 5: 'm', 6: 'i', 7: 'f', 8: 'e', 9: 'A', 10: ' ', 11: '.', 12: 's', 13: 'a', 14: 'l', 15: 'b', 16: 'h', 17: 'r', 18: 'y', 19: 'd', 20: 'o', 21: 'p', 22: 'n'}
{'S': 0, 't': 1, 'H': 2, 'c': 3, 'u': 4, 'm': 5, 'i': 6, 'f': 7, 'e': 8, 'A': 9, ' ': 10, '.': 11, 's': 12, 'a': 13, 'l': 14, 'b': 15, 'h': 16, 'r': 17, 'y': 18, 'd': 19, 'o': 20, 'p': 21, 'n': 22}


In [4]:
#time_step
time_step = 20

In [5]:
import numpy as np
from keras.utils import to_categorical
#滑动窗口提取数据
def extract_data(data, slide):    
    x = []
    y = []    
    for i in range(len(data) - slide):
        x.append([a for a in data[i:i+slide]])
        y.append(data[i+slide])        
    return x,y
#字符到数字的批量转化
def char_to_int_Data(x,y, char_to_int):    
    x_to_int = []
    y_to_int = []
    for i in range(len(x)):
        x_to_int.append([char_to_int[char] for char in x[i]])
        y_to_int.append([char_to_int[char] for char in y[i]])    
    return x_to_int, y_to_int
#实现输入字符文章的批量处理，输入整个字符、滑动窗口大小、转化字典
def data_preprocessing(data, slide, num_letters, char_to_int):    
    char_Data = extract_data(data, slide)
    int_Data = char_to_int_Data(char_Data[0], char_Data[1], char_to_int)
    Input = int_Data[0]
    Output = list(np.array(int_Data[1]).flatten())
    Input_RESHAPED = np.array(Input).reshape(len(Input), slide)
    new = np.random.randint(0,10,size=[Input_RESHAPED.shape[0],Input_RESHAPED.shape[1],num_letters])
    for i in range(Input_RESHAPED.shape[0]):
        for j in range(Input_RESHAPED.shape[1]):
            new[i,j,:] = to_categorical(Input_RESHAPED[i,j],num_classes=num_letters)
    return new, Output

Using TensorFlow backend.


In [6]:
#extract X and y from text data
X, y = data_preprocessing(data,time_step,num_letters,char_to_int)

In [7]:
print(X.shape)

(56148, 20, 23)


In [8]:
print(len(y))

56148


In [9]:
#split the data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=10)
print(X_train.shape,len(y_train))

(50533, 20, 23) 50533


In [10]:
y_train_category = to_categorical(y_train,num_letters)
print(y_train_category)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
#set up the model
from keras.models import Sequential
from keras.layers import Dense,LSTM

model = Sequential()
model.add(LSTM(units=20,input_shape=(X_train.shape[1],X_train.shape[2]),activation='relu'))
model.add(Dense(units=num_letters,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                3520      
_________________________________________________________________
dense_1 (Dense)              (None, 23)                483       
Total params: 4,003
Trainable params: 4,003
Non-trainable params: 0
_________________________________________________________________


In [13]:
#train the model
model.fit(X_train,y_train_category,batch_size=1000,epochs=5)

Epoch 1/5
50533/50533 [==============================] - 10s 195us/step - loss: 0.7781 - accuracy: 0.7737
Epoch 2/5
50533/50533 [==============================] - 7s 148us/step - loss: 1.0646 - accuracy: 0.7017
Epoch 3/5
50533/50533 [==============================] - 5s 106us/step - loss: 0.7153 - accuracy: 0.7810
Epoch 4/5
50533/50533 [==============================] - 5s 97us/step - loss: 0.3647 - accuracy: 0.9195
Epoch 5/5
50533/50533 [==============================] - 5s 105us/step - loss: 0.2583 - accuracy: 0.9514


In [14]:
#make prediction based on the training data
y_train_predict = model.predict_classes(X_train)
print(y_train_predict)

[10  8  1 ... 14  8 10]


In [15]:
#transform the int to letters
y_train_predict_char = [int_to_char[i] for i in y_train_predict]
print(y_train_predict_char)

[' ', 'e', 't', 'i', 's', 'a', 'o', ' ', ' ', 'r', 'e', 't', 'u', 'n', 'm', 'i', 'i', ' ', '.', 'd', 't', ' ', 'y', 't', 'h', 't', '.', ' ', ' ', 't', 'u', 'r', 't', ' ', ' ', 'e', ' ', 's', ' ', 's', 'h', 't', 'e', 'e', ' ', 'p', 's', 'h', 'r', 'r', 'e', 's', 'u', ' ', 'm', 'h', 't', 'r', 'e', 'i', 'h', ' ', 'l', 's', 'l', 'e', 'A', 'r', 'n', 'h', 'r', ' ', 'a', 'r', 'u', 'e', 'i', 'e', 'i', 'e', 't', 'm', 'a', 'a', 'r', ' ', 'i', ' ', 'u', 'H', 'H', 'n', 'i', 'p', 't', ' ', 't', 'f', 'o', ' ', 'o', 'S', ' ', ' ', ' ', 'p', 't', 't', ' ', 'y', 't', 'p', 'H', 'r', 'b', 'd', 'e', 's', ' ', 'h', 'r', 'h', 'n', 'e', 's', 'e', '.', 'e', 'a', ' ', 'n', 'a', 'r', 't', 'i', 't', ' ', 'n', ' ', 't', 'l', 'c', 'i', 'i', ' ', 'n', 'n', 'r', 's', 's', ' ', 't', 'e', 'l', ' ', 'r', 'u', '.', 'i', ' ', ' ', 'r', 't', 'd', 'i', 'n', 'n', 'u', 'i', 'h', 'i', 'l', 'e', 'a', 't', 'i', 't', 'l', 'A', ' ', ' ', 'c', 'l', 'i', 'r', ' ', 'e', 'i', 'e', 'i', 'h', 'a', 'd', ' ', 'a', 'i', 'm', 'a', 't', 'h',

In [16]:
from sklearn.metrics import accuracy_score
accuracy_train = accuracy_score(y_train,y_train_predict)
print(accuracy_train)

0.9583440524014011


In [17]:
y_test_predict = model.predict_classes(X_test)
accuracy_test = accuracy_score(y_test,y_test_predict)

print(accuracy_test)
print(y_test_predict)
print(y_test)


0.9547640249332146
[10 16 10 ... 13  2 14]
[10, 16, 10, 20, 4, 10, 10, 14, 12, 8, 2, 17, 9, 18, 13, 22, 8, 1, 6, 16, 10, 13, 4, 16, 6, 10, 8, 21, 22, 4, 12, 6, 9, 15, 16, 13, 3, 10, 4, 11, 7, 12, 18, 4, 2, 20, 17, 22, 9, 6, 20, 10, 10, 17, 1, 7, 10, 6, 13, 12, 1, 11, 4, 19, 6, 10, 6, 13, 6, 10, 6, 11, 2, 10, 10, 13, 20, 6, 4, 8, 8, 16, 14, 10, 6, 8, 6, 1, 8, 22, 13, 9, 17, 16, 13, 16, 17, 22, 1, 12, 10, 6, 10, 12, 8, 22, 17, 10, 11, 10, 7, 14, 10, 16, 10, 1, 6, 3, 16, 1, 10, 10, 8, 16, 12, 12, 17, 2, 13, 0, 22, 1, 17, 12, 6, 22, 19, 8, 12, 8, 6, 13, 8, 11, 4, 13, 10, 3, 12, 22, 17, 6, 1, 0, 6, 11, 20, 12, 6, 22, 16, 16, 10, 10, 4, 13, 6, 22, 19, 8, 6, 1, 12, 8, 20, 13, 12, 15, 16, 10, 8, 10, 6, 13, 12, 6, 10, 8, 6, 10, 10, 18, 8, 10, 13, 0, 14, 16, 13, 10, 13, 16, 6, 6, 10, 11, 10, 22, 6, 5, 22, 13, 10, 11, 19, 0, 8, 17, 13, 17, 17, 8, 4, 12, 10, 19, 1, 4, 6, 21, 8, 6, 16, 2, 6, 6, 9, 1, 17, 10, 13, 4, 12, 8, 22, 22, 13, 8, 10, 8, 6, 8, 13, 16, 11, 6, 6, 11, 10, 8, 8, 10, 13, 19, 13, 4

In [18]:
new_letters = 'flare is a teacher in ai industry. He obtained his phd in Australia.'
X_new, y_new = data_preprocessing(new_letters,time_step,num_letters,char_to_int)
y_new_predict = model.predict_classes(X_new)
print(y_new_predict)

[22 10 13  6 10  6 22 19  4 12  1 17 18 11 10  2  8 10 20 15  1 13  6 22
  8 19 10  6 16 21 10 21 16 19 10  6 22 10  9  4 12  1 17 13 14  6 13 11]


In [19]:
#transform the int to letters
y_new_predict_char = [int_to_char[i] for i in y_new_predict]
print(y_new_predict_char)

['n', ' ', 'a', 'i', ' ', 'i', 'n', 'd', 'u', 's', 't', 'r', 'y', '.', ' ', 'H', 'e', ' ', 'o', 'b', 't', 'a', 'i', 'n', 'e', 'd', ' ', 'i', 'h', 'p', ' ', 'p', 'h', 'd', ' ', 'i', 'n', ' ', 'A', 'u', 's', 't', 'r', 'a', 'l', 'i', 'a', '.']


In [21]:
for i in range(0,X_new.shape[0]-20):
    print(new_letters[i:i+20],'--predict next letter is---',y_new_predict_char[i])

flare is a teacher i --predict next letter is--- n
lare is a teacher in --predict next letter is---  
are is a teacher in  --predict next letter is--- a
re is a teacher in a --predict next letter is--- i
e is a teacher in ai --predict next letter is---  
 is a teacher in ai  --predict next letter is--- i
is a teacher in ai i --predict next letter is--- n
s a teacher in ai in --predict next letter is--- d
 a teacher in ai ind --predict next letter is--- u
a teacher in ai indu --predict next letter is--- s
 teacher in ai indus --predict next letter is--- t
teacher in ai indust --predict next letter is--- r
eacher in ai industr --predict next letter is--- y
acher in ai industry --predict next letter is--- .
cher in ai industry. --predict next letter is---  
her in ai industry.  --predict next letter is--- H
er in ai industry. H --predict next letter is--- e
r in ai industry. He --predict next letter is---  
 in ai industry. He  --predict next letter is--- o
in ai industry. He o --predict 